In [3]:
import numpy as np
import pandas as pd
from glob import glob
import xml.etree.ElementTree as xet
import cv2
import os

In [4]:
df = pd.read_csv('../labels.csv')
df.head()

,filepath,xmin,xmax,ymin,ymax
0,./images_labeled_xml\N1.xml,1093,1396,645,727
1,./images_labeled_xml\N100.xml,134,301,312,350
2,./images_labeled_xml\N101.xml,31,139,128,161
3,./images_labeled_xml\N102.xml,164,316,216,243
4,./images_labeled_xml\N103.xml,813,1067,665,724


#### Labels YOLO

center_x, center_y, width , height

In [5]:
def parsing(path):
    path = f".{path}"
    parser = xet.parse(path).getroot()
    name = parser.find('filename').text
    filename = f'../images_labeled_xml/{name}'

    parser_size = parser.find('size')
    width = int(parser_size.find('width').text)
    height = int(parser_size.find('height').text)
    
    return filename, width, height

df[['filename','width','height']] = df['filepath'].apply(parsing).apply(pd.Series)

In [6]:
df.head()

,filepath,xmin,xmax,ymin,ymax,filename,width,height
0,./images_labeled_xml\N1.xml,1093,1396,645,727,../images_labeled_xml/N1.jpeg,1920,1080
1,./images_labeled_xml\N100.xml,134,301,312,350,../images_labeled_xml/N100.jpeg,450,417
2,./images_labeled_xml\N101.xml,31,139,128,161,../images_labeled_xml/N101.jpeg,249,239
3,./images_labeled_xml\N102.xml,164,316,216,243,../images_labeled_xml/N102.jpeg,478,395
4,./images_labeled_xml\N103.xml,813,1067,665,724,../images_labeled_xml/N103.jpeg,1800,1200


In [7]:
# center_x, center_y, width , height
df['center_x'] = (df['xmax'] + df['xmin'])/(2*df['width'])
df['center_y'] = (df['ymax'] + df['ymin'])/(2*df['height'])

df['bb_width'] = (df['xmax'] - df['xmin'])/df['width']
df['bb_height'] = (df['ymax'] - df['ymin'])/df['height']

In [8]:
df.head()

,filepath,xmin,xmax,ymin,ymax,filename,width,height,center_x,center_y,bb_width,bb_height
0,./images_labeled_xml\N1.xml,1093,1396,645,727,../images_labeled_xml/N1.jpeg,1920,1080,0.648177,0.635185,0.157812,0.075926
1,./images_labeled_xml\N100.xml,134,301,312,350,../images_labeled_xml/N100.jpeg,450,417,0.483333,0.793765,0.371111,0.091127
2,./images_labeled_xml\N101.xml,31,139,128,161,../images_labeled_xml/N101.jpeg,249,239,0.341365,0.604603,0.433735,0.138075
3,./images_labeled_xml\N102.xml,164,316,216,243,../images_labeled_xml/N102.jpeg,478,395,0.502092,0.581013,0.317992,0.068354
4,./images_labeled_xml\N103.xml,813,1067,665,724,../images_labeled_xml/N103.jpeg,1800,1200,0.522222,0.578750,0.141111,0.049167


In [9]:
### Split the data into train and test
df_train = df.iloc[:200]
df_test = df.iloc[200:]

text file (labels)

class_id, center_x, center_y, bb_width, bb_height

In [10]:
from shutil import copy

In [12]:
train_folder = './data_images/train'

values = df_train[['filename','center_x','center_y','bb_width','bb_height']].values
for fname, x,y, w, h in values:
    image_name = os.path.split(fname)[-1]
    txt_name = os.path.splitext(image_name)[0]
    
    dst_image_path = os.path.join(train_folder,image_name)
    dst_label_file = os.path.join(train_folder,txt_name+'.txt')
    
    # copy each image into the folder
    copy(fname,dst_image_path)

    # generate .txt which has label info
    label_txt = f'0 {x} {y} {w} {h}'
    with open(dst_label_file,mode='w') as f:
        f.write(label_txt)
        
        f.close()


In [13]:
test_folder = './data_images/test'

values = df_test[['filename','center_x','center_y','bb_width','bb_height']].values
for fname, x,y, w, h in values:
    image_name = os.path.split(fname)[-1]
    txt_name = os.path.splitext(image_name)[0]
    
    dst_image_path = os.path.join(test_folder,image_name)
    dst_label_file = os.path.join(test_folder,txt_name+'.txt')
    
    # copy each image into the folder
    copy(fname,dst_image_path)

    # generate .txt which has label info
    label_txt = f'0 {x} {y} {w} {h}'
    with open(dst_label_file,mode='w') as f:
        f.write(label_txt)
        
        f.close()


---